In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import qa_questions
import random
import joblib
from tqdm import tqdm
import feature_spaces
fit_encoding = __import__('01_fit_encoding')

### Look at examples

In [ ]:
class A:
    use_test_setup = False
    subject = 'UTS03'


args = A()

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)

In [ ]:
wordseqs = feature_spaces.get_story_wordseqs(story_names_train)
# chunks = wordseqs['theclosetthatateeverything'].chunks()
# chunk_lens = [len(chunk) for chunk in chunks]

In [41]:
seed = 43  # 42
ngrams_examples = []
ngram_size = 10
num_examples_per_story = 1
random.seed(seed)
np.random.seed(seed)
for story_name in story_names_train:
    words_list = wordseqs[story_name].data
    ngrams_list = feature_spaces.get_ngrams_list_from_words_list(
        words_list, ngram_size=ngram_size)[ngram_size + 2:]
    ngrams_examples += np.random.choice(ngrams_list,
                                        num_examples_per_story).tolist()

In [42]:
print('\n'.join(['- ' + ngram for ngram in ngrams_examples]))

- of them we took them to the recycling bin and the
- a helpless victim of a sickness defined by a condition and
- you sure you're gonna just add lesbian to that list like
- them said i'm going to shoot the dog and the other
- take all our kids and go cross country and on the
- boyfriend and he answers so i put a pound coin in
- had blood running down my arms and all over my little
- hours thanks in part to the addition of some lighter fluid
- doing the doctor's advice i enrolled to do actuarial science at
- subject is in school and tell him you know why i
- to the movies i pick them up from school in the
- voicemail on my phone from a man who identifies himself as
- wasn't good but somehow i managed to get through high school
- of leukemia in october of nineteen fifty five there is a
- didn't wanna be so logical i actually wanted something more magic
- gifts and clothes because that usually means going somewhere new and
- him if i had a cat scan he said no and
- by now it's around fiv

In [ ]:
ngrams_list[:10]

In [ ]:
args

### Load fitted models

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/results_mar30'
experiment_filename = '../experiments/01_fit_encoding.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)
# imodelsx.process_results.delete_runs_in_dataframe(
# r[r.feature_space == 'qa_embedder-10'], actually_delete=True)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)

In [ ]:
args = r[(r.feature_space == 'qa_embedder-5') * (r.pc_components == 100) * (r.ndelays == 4)
         ].sort_values(by='corrs_tune_mean', ascending=False).iloc[0]

In [ ]:
args['corrs_test_pc']

In [ ]:
story_names_train, story_names_test = fit_encoding.get_story_names(args)
stim_test_delayed, resp_test = fit_encoding.get_data(args, story_names_test)
stim_train_delayed, resp_train = fit_encoding.get_data(args, story_names_train)